In [1]:
from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
import time
import pickle

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 4*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------

from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp
import sys
current = os.getcwd()
parent_directory = os.path.dirname(current)
sys.path.append(parent_directory)
from ICET_spherical import ICET
from utils import *
from metpy.calc import lat_lon_grid_deltas
from pioneer.das.api.platform import Platform
from scipy.spatial.transform import Rotation as R
from pioneer.das.api.egomotion.imu_egomotion_provider import IMUEgomotionProvider as emp 
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook

2023-04-30 21:20:09.886935: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 21:20:10.531104: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-30 21:20:11.673132: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/derm/anaconda3/envs/py39/lib/python3.9/site-packages/cv2/../../lib64:
2023-04-30 21:20:11.673227: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_p

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-04-30 21:20:13.616399: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 21:20:13.617785: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-30 21:20:13.617948: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-30 21:20:13.618075: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Autosaving every 180 seconds


# Goal: Simultaneously Estimate Translation, Rotation, and Distortion

## $\mathbf{A} = [\hat{X}_{ij}, \hat{m}_{ij}] = 
\begin{bmatrix}
% x, ~ y, ~ z, ~ \phi, ~ \theta, ~ \psi, ~ x^+, ~y^+, ~z^+, ~\phi^+, ~\theta^+, ~\psi^+ \\
x, ~ y, ~ z, ~ \phi, ~ \theta, ~ \psi, ~ x^+, ~y^+, ~z^+, ~\phi^+, ~\theta^+, ~\psi^+ \\
\end{bmatrix}$

We can use newton-raphson to find A

<!-- ## $y_i = \mathbf{h}(y_j, \hat{X}_{ij}, \hat{m}_{ij}) + \mathbf{H}_m \delta m + \mathbf{H}_x \delta x + \text{H.O.T.}$ -->

## $y_i = \mathbf{h}(y_j, \hat{A}_{ij}) + \mathbf{H}_A \delta A + \text{H.O.T.}$

## $\mathbf{H}_A \in \mathbb{R}^{4N \times 12} $



### $\mathbf{H}_A = [H_X, H_m]$

# Run Demo

In [6]:
# load point cloud
# # no distortion
# old_cloud =  np.load("sample_data/paper_figures/case1/raw_frame_0.npy")
# # m_hat = np.array([0., 0., 0., 0., 0., 0.0]) #actual motion
# m_hat = np.array([-3., 0., 0., 0., 0., 0.1]) #test wrap around

# movement in x
old_cloud =  np.load("sample_data/paper_figures/case2/raw_frame_3.npy") 
m_hat = np.array([3, 0, 0., 0., 0., 0])
# m_hat = np.array([3, 0, 0., 0., 0., -0.2]) #FOR DEBUG-- deform just a little
gt =  np.load("sample_data/paper_figures/case2/base_vel_2.npy")

# # movement in x, y, & yaw
# old_cloud =  np.load("sample_data/paper_figures/case3/raw_frame_1.npy") 
# # m_hat = np.array([3, -1, 0., 0., 0., -1])
# m_hat = np.array([3, -1, 0., 0., 0., -0.86]) #FOR DEBUG-- deform a little extra
# # m_hat = np.array([0., 0., 0., 0., 0., 0.0]) #FOR DEBUG-- no deformation
# gt =  np.load("sample_data/paper_figures/case3/base_vel_2.npy")
# # print(gt) 

# period_lidar = 1
# t_scale = (2*np.pi)/(-m_hat[-1] + (2*np.pi/period_lidar))
# print(t_scale)
# m_hat = m_hat*t_scale
# # m_hat[-1] = m_hat[-1]*t_scaled
# print(m_hat)

#downsample
old_cloud = old_cloud[::5,:]

In [13]:
from remove_motion_basic import linear_correction_old as lc 

#apply ground truth distortion according to m_hat
new_cloud = lc(old_cloud, m_hat) 
#set ground truth transform between clouds
# X_gt = np.array([0, 0., 0.0, 0.0, 0.0, 0.0])
# X_gt = np.array([1.5, 0.5, 0.03, 0.03, 0.03, 0.25])
# X_gt = np.array([1.5, -3, 0.1, 0.2, 0.03, -0.15])
X_gt = np.array([-3, 0., 0.0, 0.0, 0.0, 0.0])

# add noise
old_cloud += 0.01*np.random.randn(np.shape(old_cloud)[0], 3)

# #for fig: mess up both again to show that relative distortion remains constant
# new_cloud = lc(new_cloud, -m_hat)
# old_cloud = lc(old_cloud, -m_hat)

#remove ground plane
old_cloud = old_cloud[old_cloud[:,2] > -1] 
new_cloud = new_cloud[new_cloud[:,2] > -1] 

# Rotate + Translate new point cloud
trans = X_gt[:3]
rot = R_tf(X_gt[3:]).numpy()
new_cloud = (new_cloud @ rot) + trans

plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
disp.append(Points(old_cloud, c = "#CB2314")) 
disp.append(Points(new_cloud, c = "#2c7c94")) 
plt.show(disp, "raw point clouds")
ViewInteractiveWidget(plt.window)

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

### Attempt to solve with basic 6 state solution (Impossible)

In [14]:
from linear_corrector import LC
pc1 = old_cloud
pc2  = new_cloud
m_hat0 = np.array([0, 0, 0, 0, 0, 0.])
dc = LC(cloud1 = pc2, cloud2 = pc1, fid = 50, niter = 20, draw = True, m_hat0 = m_hat0,  mnp = 25, RM = False)
ViewInteractiveWidget(dc.plt.window)


~~~~~~~~~~~Iteration  0 ~~~~~~~~~~
took 0.017620086669921875 sec  to apply motion profile

 M 
 (73, 6)
H_m 
 (292, 6)
took 0.0018317699432373047 sec to get H
m_hat:  [-0.20490022 -0.00844516 -0.26121098  0.00464209 -0.03701815 -0.00690006]
~~~~~~~~~~~Iteration  1 ~~~~~~~~~~
took 0.015044212341308594 sec  to apply motion profile

 M 
 (79, 6)
H_m 
 (316, 6)
took 0.0018990039825439453 sec to get H
m_hat:  [-0.22695127 -0.02058472 -0.37419638 -0.00845655 -0.02406741 -0.00726894]
~~~~~~~~~~~Iteration  2 ~~~~~~~~~~
took 0.014275074005126953 sec  to apply motion profile

 M 
 (79, 6)
H_m 
 (316, 6)
took 0.0020647048950195312 sec to get H
m_hat:  [-0.22258125 -0.0192147  -0.44036609 -0.00507331 -0.03048321 -0.00752197]
~~~~~~~~~~~Iteration  3 ~~~~~~~~~~
took 0.014877796173095703 sec  to apply motion profile

 M 
 (78, 6)
H_m 
 (312, 6)
took 0.0023348331451416016 sec to get H
m_hat:  [-0.22706306 -0.02151241 -0.48967582 -0.00841068 -0.0295359  -0.0074128 ]
~~~~~~~~~~~Iteration  4 ~~~~~~~~~~
t

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

In [21]:
# ##DEBUG: make sure that correspondeces between old and new cloud are good
# pc1_theta = dc.c2s(pc1)[:,1]
# pc2_theta = dc.c2s(pc2)[:,1]
# from matplotlib import pyplot as plt
# fig, ax = plt.subplots()
# ax.plot(pc1_theta)
# ax.plot(pc2_theta)

In [28]:
A_hat = np.array([0., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
skip = 1 #50
y_i = new_cloud[::skip] #baseline
y_j = old_cloud[::skip] #distorted cloud

plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
disp.append(Points(y_i[:,:3], c = "#2c7c94 ", alpha = 0.5, r=5.5))

runlen = 5
for count in range(runlen):
    
    print("~~~~ iteration ", count, "~~~~~~~~~~~")
    print("A_hat = \n", np.round(A_hat[:6],4), "\n", np.round(A_hat[6:],4)) 
    
    #decompose A_hat into X_hat and m_hat
    X_hat = A_hat[:6] 
    m_hat = A_hat[6:]     
#     X_hat = np.array([0, 0, 0, 0, 0, 0.]) #for debug
#     m_hat = np.array([0, 0, 0, 0, 0, 0.]) #for debug
#     m_hat = 0.1*m_hat
    
    #apply last estimate of distortion correction
#     print("m_hat", m_hat)
    y_j_undistort = lc(y_j, m_hat)

    #apply last rigid transform
    rot = R_tf(X_hat[3:]).numpy()
    trans = X_hat[:3]
    y_j_undistort = (y_j_undistort @ rot) + trans
    
#     print("rot: \n", rot,"\n trans: \n", trans)  
#     print("\n y_i \n",np.shape(y_i), "\n", y_i[:3])
#     print("y_j_undistort \n",np.shape(y_j_undistort), "\n", y_j_undistort[:3])

    
    #get H = [H_X, H_m]
    #get jacobain of distortion correction function
    H_m = dc.get_H_m(y_j_undistort, m_hat) 
#     print("\n H_m:", np.shape(H_m), "\n", H_m[:10])
    
    #get jacobian of rigid transform function 
    H_x = jacobian_tf(tf.transpose(tf.convert_to_tensor(y_j_undistort, tf.float32)), tf.convert_to_tensor(X_hat[3:], tf.float32)) # shape = [num of corr * 3, 6]
    #need to append on a row of zeros since we are working with homogeneous coordinates!
    H_x = tf.reshape(H_x, (tf.shape(H_x)[0]//3, 3, 6)) # -> need shape [#corr//4, 4, 6]
#     print("\n H_x:", np.shape(H_x), "\n", H_x[0])
    H_x = tf.concat([H_x, tf.zeros([len(H_x),1,6])], axis = 1)
    H_x = tf.reshape(H_x, (-1, 6))
#     print("\n H_x:", np.shape(H_x), "\n", H_x[:10])
    H_x = H_x.numpy()
        
    #delta_A =  ((H^T*H)^-1)(H^T)(yi - yj_undistort)
    residual = (np.append(y_i, np.ones([len(y_i),1]), axis = 1) -
                np.append(y_j_undistort, np.ones([len(y_i),1]), axis = 1)).flatten()
#     print("residual", np.shape(residual), "\n", residual)
    
    H = np.append(H_x, H_m, axis = 1)
#     print("H: \n", np.shape(H))
    
    delta_A = np.linalg.pinv(H.T @ H) @ H.T @ residual
    print("\n delta_A \n", np.round(delta_A[:6], 5), "\n", np.round(delta_A[6:], 5))
    #augment rigid transform components
    A_hat[:6] -=   delta_A[:6]
    #augment distortion components
    A_hat[6:9] -= delta_A[6:9]
    A_hat[9:] += delta_A[9:]

    #plot updated cloud2
#     color = [0.5 + count/(runlen*2), 1 - (count+1)/runlen, (count+1)/runlen]
#     disp.append(Points(y_j_undistort[:,:3], c = color, r=3.5))
    disp.append(Points(y_j_undistort[:,:3], c = "#a65852 ", alpha = (count+1)/(runlen+1), r=3.5))

    
plt.show(disp, "12 State Solution")
ViewInteractiveWidget(plt.window)

~~~~ iteration  0 ~~~~~~~~~~~
A_hat = 
 [0. 0. 0. 0. 0. 0.] 
 [0. 0. 0. 0. 0. 0.]

 M 
 (18074, 6)
H_m 
 (72296, 6)

 delta_A 
 [ 1.32669e+00  3.43000e-03  2.87700e-02 -9.70000e-04  3.71000e-03
 -1.13000e-03] 
 [-2.66862e+00 -3.20000e-03 -6.54600e-02 -3.40000e-04  7.99000e-03
 -8.70000e-04]
~~~~ iteration  1 ~~~~~~~~~~~
A_hat = 
 [-1.3267e+00 -3.4000e-03 -2.8800e-02  1.0000e-03 -3.7000e-03  1.1000e-03] 
 [ 2.6686e+00  3.2000e-03  6.5500e-02 -3.0000e-04  8.0000e-03 -9.0000e-04]

 M 
 (18074, 6)
H_m 
 (72296, 6)

 delta_A 
 [ 0.15818 -0.00155 -0.0154   0.0009  -0.00264  0.00105] 
 [-0.29733  0.00189  0.03588  0.0006  -0.0061   0.00083]
~~~~ iteration  2 ~~~~~~~~~~~
A_hat = 
 [-1.4849e+00 -1.9000e-03 -1.3400e-02  1.0000e-04 -1.1000e-03  1.0000e-04] 
 [ 2.9659e+00  1.3000e-03  2.9600e-02  3.0000e-04  1.9000e-03 -0.0000e+00]

 M 
 (18074, 6)
H_m 
 (72296, 6)

 delta_A 
 [ 1.393e-02 -1.470e-03 -8.640e-03  1.200e-04 -6.200e-04  9.000e-05] 
 [-3.122e-02  7.300e-04  1.925e-02 -3.000e-05 -1.140e

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)